In [288]:
import requests
from requests.auth import HTTPBasicAuth
import io
import zipfile
import pandas as pd
import re
import os
import datetime
import numpy as np
import dateutil
import sys
import time
import matplotlib.pylab as plt
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import exc
import urllib
import pyodbc
import json
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'}

In [307]:
def create_table(engine, database_name, sql_statement):
    c = False
    while c == False:
        
        try:
            engine.execute('USE'+' '+str(database_name))
            a = engine.execute(str(sql_statement))
            if a:
                print('created table'+' '+str(sql_statement))
            break
            
        except:
            
            print('cant create table'+' '+str(sql_statement))
            break 

In [308]:
def drop_table(engine, database_name, sql_statement):
    c = False
    while c == False:
        
        try:
            engine.execute('USE'+' '+str(database_name))
            a = engine.execute(str(sql_statement))
            if a:
                print('droped table'+' '+str(sql_statement))
            break
            
        except:
            
            print('cant drop table'+' '+str(sql_statement))
            break 

In [289]:
#import database password
with open('database.json', 'r') as f:
    config = json.load(f)
    
password = config[0]['password']
user_name = config[0]['user_id']
database = config[0]['database']
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE="+str(database)+";UID="+str(user_name)+";PWD="+str(password)

params = urllib.parse.quote_plus(connection_string)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
connection=engine.connect()

In [290]:
#statement = 'SELECT prod.ProductID, prod.ProductName, prod.Price, prod.ProductDescription FROM dbo.Products as prod;'
#df = pd.read_sql(statement,cnxn)
#df

In [291]:
def get_codes(test):
    
        country_dict = {'Australia':'al',
                'Austria':'au',
                'Belgium':'be',
                'Brazil':'bz',
                'Canada':'ca',
                'China, P.R.':'ch',
                'Denmark':'dn',
                'EMU member countries':'eu',
                'European Union':'ec',
                'Finland':'fn',
                'France':'fr',
                'Germany':'ge',
                'Greece':'gr',
                'Hong Kong':'hk',
                'India':'in',
                'Ireland':'ir',
                'Italy':'it',
                'Japan':'ja',
                'Malaysia':'ma',
                'Mexico':'mx',
                'Netherlands':'ne',
                'New Zealand':'nz',
                'Norway':'no',
                'Portugual':'po',
                'Singapore':'si',
                'South Africa':'sf',
                'South Korea':'ko',
                'Spain':'sp',
                'Sri Lanka':'sl',
                'Sweden':'sd',
                'Switzerland':'sz',
                'Taiwan':'ta',
                'Thailand':'th',
                'United Kingdom':'uk',
                'Venezuela':'ve'}
    

        
        s = pd.Series(country_dict)
        df_code = pd.DataFrame({'Country':s.index, 'code':s.values})
        if test:
            df_code = df_code.sample(n=1)
        return(df_code)


In [292]:
def get_structure(url_l, code_frame):
    lis = []
    
    try:
        
        for r in url_l:
        
            if r != 'https://www.federalreserve.gov/releases/h10/hist/default.htm':
                df_exchange = pd.read_html(r,header = 0)[2]
                df_exchange = df_exchange.drop([0, 0])
                df_exchange['Country'] = df_exchange['Country or region'].str.replace(r"\(.*\)","")
                df_exchange['Country'] = df_exchange['Country'].str.rstrip() 
                df_exchange['Contains data'] = df_exchange['Country or region'].str.contains('no data', na=False)
                contains_data = [not x for x in df_exchange['Contains data']]
                df_exchange = df_exchange[contains_data]
                df_exchange = pd.merge(df_exchange,code_frame, left_on = 'Country', right_on = 'Country')
                df_exchange['year'] = url_l[r]
                print(df_exchange)
                lis.append(df_exchange)
            else:
                df_exchange = pd.read_html(r,header = 0)[0]
                df_exchange = df_exchange.drop([0, 0])
                df_exchange['Country'] = df_exchange['Country'].str.replace(r"\(.*\)","")
                df_exchange['Country'] = df_exchange['Country'].str.rstrip() 
                df_exchange['Contains data'] = df_exchange['Country'].str.contains('no data', na=False)
                contains_data = [not x for x in df_exchange['Contains data']]
                df_exchange = df_exchange[contains_data]
                df_exchange = pd.merge(df_exchange,code_frame, left_on = 'Country', right_on = 'Country')
                df_exchange['year'] = url_l[r]
                print(df_exchange)
                lis.append(df_exchange)
                
        return(lis)
    
    except:
        print('Cant get detail for'+ str(r))
        pass

In [293]:
def get_structure_list(url_l, code_frame):
    lis = []
    try:
        #r is a dictionary
        for r in url_l:
            
            for key, value in r.items():
        
                if key != 'https://www.federalreserve.gov/releases/h10/hist/default.htm':
                    df_exchange = pd.read_html(key,header = 0)[2]
                    df_exchange = df_exchange.drop([0, 0])
                    df_exchange['Country'] = df_exchange['Country or region'].str.replace(r"\(.*\)","")
                    df_exchange['Country'] = df_exchange['Country'].str.rstrip() 
                    df_exchange['Contains data'] = df_exchange['Country or region'].str.contains('no data', na=False)
                    contains_data = [not x for x in df_exchange['Contains data']]
                    df_exchange = df_exchange[contains_data]
                    df_exchange = pd.merge(df_exchange,code_frame, left_on = 'Country', right_on = 'Country')
                    df_exchange['year'] = value
                    lis.append(df_exchange)
                else:
                    df_exchange = pd.read_html(key,header = 0)[0]
                    df_exchange = df_exchange.drop([0, 0])
                    df_exchange['Country'] = df_exchange['Country'].str.replace(r"\(.*\)","")
                    df_exchange['Country'] = df_exchange['Country'].str.rstrip() 
                    df_exchange['Contains data'] = df_exchange['Country'].str.contains('no data', na=False)
                    contains_data = [not x for x in df_exchange['Contains data']]
                    df_exchange = df_exchange[contains_data]
                    df_exchange = pd.merge(df_exchange,code_frame, left_on = 'Country', right_on = 'Country')
                    df_exchange['year'] = value
                    lis.append(df_exchange)
            
            
        return(lis)
    
    except:
        print('Cant get detail for'+ str(r))
        pass

In [294]:
def exchange_list(df_list, base_url):
    exchange_data = []
    url_ext = base_url
    for df in df_list:
        
        try:
     
            for x,m,c,y in zip(df['code'],df['Monetary unit'],df['Country'],df['year']):
                url_ext = base_url.replace('YYYY',str(y)) # add the date to the end of url
                url_ext = url_ext.replace('CC', str(x))       
                r = requests.get(url_ext, allow_redirects=True, stream=True, headers=headers)
                if r.status_code == requests.codes.ok:
                    df_ex = pd.read_html(url_ext, header = 0)[0]
                    df_ex['Monetary Unit'] = str(m)
                    df_ex['Country'] = str(c)
                    #print(df_ex.head())
                    exchange_data.append(df_ex)
                
                else:
                    print('couldnt get data for' +str(x)+' '+str(y))
                    
        except:
            print('Cant get rates for'+' '+str(c)+' '+str(url_ext))
            continue
                
    exchange = pd.concat(exchange_data, axis=0, sort=False, ignore_index=True)
    exchange = exchange.dropna(axis=0)
    
    return(exchange)

In [295]:
def get_rates(test, historical):
    
    url_list = [{'https://www.federalreserve.gov/releases/h10/hist/default1989.htm':'89'},
                {'https://www.federalreserve.gov/releases/h10/hist/default1999.htm':'96'},
                {'https://www.federalreserve.gov/releases/h10/hist/default.htm':'00'}]


    url = 'https://www.federalreserve.gov/releases/h10/hist/datYYYY_CC.htm' #base url
    
    if historical:
        #print(url_list)
        pass
    else:
        #print([url_list[-1]])
        url_list = [url_list[-1]]
    
    if test:
        df_detail = get_codes(test=True)
        #print(df_detail)
    else:
        df_detail = get_codes(test=False)
        #print(df_detail)
        
    #df = get_structure(url_dict, df_detail)
    #change data structure!
    df = get_structure_list(url_list, df_detail)
    federal_reserve = exchange_list(df, url)
    federal_reserve.replace('ND', np.nan, inplace=True)
    federal_reserve.replace('NC', np.nan, inplace=True)
    federal_reserve['Date'] = pd.to_datetime(federal_reserve['Date'])
    federal_reserve['Rate'] = federal_reserve['Rate'].astype('float64')
    federal_reserve['Monetary Unit'] = federal_reserve['Monetary Unit'].astype('object')
    federal_reserve['Country'] = federal_reserve['Country'].astype('object')
    federal_reserve['Monetary Unit'] = federal_reserve['Monetary Unit'].astype('object')
    #add the columns column later for doing analysis and creating the unpivoted column headings
    federal_reserve['Columns'] = federal_reserve['Country']+'-'+federal_reserve['Monetary Unit']
    
    return(federal_reserve)

In [311]:
#drop the table if neccecary
#sql_drop = 'DROP TABLE IF EXISTS rates'
#drop_table(connection,'federal_reserve',sql_drop)
#create the table if neccecary
#sql_create = 'CREATE TABLE rates (Date DATE, Rate FLOAT, [Monetary Unit] VARCHAR(50), Country VARCHAR(50), Columns VARCHAR(75));'
#create_table(connection,'federal_reserve',sql_create)

In [333]:
s = 'SELECT * from rates'
database_rates = pd.read_sql_query(s,connection)
database_rates['Date'] = pd.to_datetime(database_rates['Date'])

In [334]:
database_rates.tail()

,Date,Rate,Monetary Unit,Country,Columns
338193,2019-02-18,NaN,Bolivar,Venezuela,Venezuela-Bolivar
338194,2019-02-19,3289.9645,Bolivar,Venezuela,Venezuela-Bolivar
338195,2019-02-20,NaN,Bolivar,Venezuela,Venezuela-Bolivar
338196,2019-02-21,3290.3934,Bolivar,Venezuela,Venezuela-Bolivar
338197,2019-02-22,3290.6028,Bolivar,Venezuela,Venezuela-Bolivar


In [336]:
#function to determine if scraping all the data is neccecary:
def scrape(database,test_years):
    years = [x.year for x in database['Date']]
    
    test_years = [1989,1996]

    if test_years[0] and test_years[1] in years:
        return(False)
    else:
        return(True)

In [328]:
#scrape the rates table
scrape_historical = scrape(database_rates,[1989,1996])
rates = get_rates(test=False, historical = scrape_historical)

In [337]:
rates.head()

,Date,Rate,Monetary Unit,Country,Columns
0,2000-01-04,0.6562,Dollar,Australia,Australia-Dollar
1,2000-01-05,0.6550,Dollar,Australia,Australia-Dollar
2,2000-01-06,0.6540,Dollar,Australia,Australia-Dollar
3,2000-01-07,0.6548,Dollar,Australia,Australia-Dollar
4,2000-01-10,0.6561,Dollar,Australia,Australia-Dollar


In [338]:
#function to return all of the scraped data that is not in the database
def return_not_in_database(df1, df2):
    #intersected_df = pd.merge(database_df, scrape_df, how='inner')
    
    
    intersected_df = pd.merge(df1, df2, how='right', indicator=True)
    intersected_df = intersected_df[intersected_df['_merge']=='right_only']
    intersected_df = intersected_df.drop('_merge',1)
    return(intersected_df)

In [339]:
insert_to_sql = return_not_in_database(df1 = database_rates,df2 = rates)
insert_to_sql.head()

,Date,Rate,Monetary Unit,Country,Columns


In [340]:
sql_table = 'rates'
insert_to_sql.to_sql(sql_table, connection, if_exists='append', index=False,chunksize=100)

In [27]:
def pivoter(df):
    #unpivot = df.pivot('Date', columns='Country', values='Rate')
    unpivot = pd.pivot_table(df,values='Rate',index='Date',columns='Columns')
    return(unpivot)

In [28]:
rates_unpivot = pivoter(rates)
#u.dropna(axis=0,inplace=True)
c=rates_unpivot.corr(method='pearson')

In [29]:
rates_unpivot.tail()

Columns,New Zealand-Dollar
Date,
2019-02-11,0.6732
2019-02-12,0.6740
2019-02-13,0.6817
2019-02-14,0.6829
2019-02-15,0.6856


In [11]:
c

Columns,Australia-Dollar,"China, P.R.-Yuan",European Union-Ecu,New Zealand-Dollar
Columns,,,,
Australia-Dollar,1.000000,-0.467156,0.171111,0.943983
"China, P.R.-Yuan",-0.467156,1.000000,0.581783,-0.151007
European Union-Ecu,0.171111,0.581783,1.000000,0.437363
New Zealand-Dollar,0.943983,-0.151007,0.437363,1.000000


In [12]:
price_dict = {'WTI':'https://www.eia.gov/dnav/pet/hist_xls/RWTCd.xls',
              'Brent':'https://www.eia.gov/dnav/pet/hist_xls/RBRTEd.xls',
              'Henry Hub':'https://www.eia.gov/dnav/ng/hist_xls/RNGWHHDd.xls'}

In [13]:
#Get oil price data from eia
def download_file(url):

    r = requests.get(url, allow_redirects=True, stream=True, headers=headers)
    if r.status_code == requests.codes.ok:
         df = pd.read_excel(io.BytesIO(r.content), sheet_name='Data 1', 
                        header=2, parse_dates=['Date'])
    else:
        print('Error! File could not be downloaded from: ' + url)
        
    return(df)

In [14]:
def get_eia(dowload_func,price_dict):
    eia_prices = {}

    eia_frame = pd.DataFrame()

    for key in price_dict:
        left = key
        right = price_dict[key]
        right = download_file(right)
        eia_prices.update({left:right})
    
    for key in eia_prices:
    
        join1 = eia_prices[key].copy()
    
        if eia_frame.empty:
            eia_frame = join1
        else:
            eia_frame = pd.merge(eia_frame,join1, how='outer',left_on='Date', right_on='Date')

    eia_frame = eia_frame.sort_values(by=['Date'])
    return(eia_frame)

In [15]:
r = get_eia(download_file,price_dict)

In [16]:
#v = price_rates.corr()

In [17]:
def join_rates_prices(rates,prices):
    price_rates = pd.merge(prices,rates, how = 'left', left_on='Date',right_on='Date',sort=True)
    price_rates.set_index('Date',inplace=True)
    return(price_rates)

In [18]:
v = join_rates_prices(rates_unpivot,r)

In [19]:
v

,"Cushing, OK WTI Spot Price FOB (Dollars per Barrel)",Europe Brent Spot Price FOB (Dollars per Barrel),Henry Hub Natural Gas Spot Price (Dollars per Million Btu),Australia-Dollar,"China, P.R.-Yuan",European Union-Ecu,New Zealand-Dollar
Date,,,,,,,
1986-01-02,25.56,NaN,NaN,0.6820,3.2095,0.8911,0.5020
1986-01-03,26.00,NaN,NaN,0.6810,3.2095,0.8876,0.5005
1986-01-06,26.53,NaN,NaN,0.6807,3.2095,0.8846,0.5240
1986-01-07,25.85,NaN,NaN,0.6885,3.2095,0.8968,0.5200
1986-01-08,25.87,NaN,NaN,0.6886,3.2095,0.8934,0.4980
1986-01-09,26.03,NaN,NaN,0.6965,3.2095,0.8871,0.5070
1986-01-10,25.65,NaN,NaN,0.6956,3.2095,0.8898,0.5050
1986-01-13,25.08,NaN,NaN,0.6938,3.2095,0.8822,0.5030
1986-01-14,24.97,NaN,NaN,0.6911,3.2095,0.8826,0.5045


In [20]:
def unique_combinations(df):
    
    col = v.columns
    col = list(col)
    u_d = {}
    combinations = {}
    i = 0
    lis = []

    for r in col:
        for t in col:
            if r!=t:
                i=i+1
                combinations.update({i:{r:t}})

            
    c = 0
    for key in combinations.values():
        x = key
        res = dict((v,k) for k,v in x.items())
        #if not lis:
        #    lis.append(x)
        if x and res not in lis:
            lis.append(x)
            
    return(lis)

In [21]:
f = unique_combinations(v)

In [22]:
f

[{'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)': 'Europe Brent Spot Price FOB (Dollars per Barrel)'},
 {'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)': 'Henry Hub Natural Gas Spot Price (Dollars per Million Btu)'},
 {'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)': 'Australia-Dollar'},
 {'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)': 'China, P.R.-Yuan'},
 {'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)': 'European Union-Ecu'},
 {'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)': 'New Zealand-Dollar'},
 {'Europe Brent Spot Price FOB (Dollars per Barrel)': 'Henry Hub Natural Gas Spot Price (Dollars per Million Btu)'},
 {'Europe Brent Spot Price FOB (Dollars per Barrel)': 'Australia-Dollar'},
 {'Europe Brent Spot Price FOB (Dollars per Barrel)': 'China, P.R.-Yuan'},
 {'Europe Brent Spot Price FOB (Dollars per Barrel)': 'European Union-Ecu'},
 {'Europe Brent Spot Price FOB (Dollars per Barrel)': 'New Zealand-Dollar'},
 {'Henry Hub Natural Gas Spot Pri

In [23]:
#take the unique combinations and count the number of nan and number of real data
#real data should be the number of times that there is a value for both datasets

def count_na(combinations):
    
    for x in combinations:
        
        for key in x:
        
            df_new = v[[key,x[key]]]
            count_na = df_new[key].isna().sum()
            count_na2 = df_new[x[key]].isna().sum()
            number_na = max(count_na,count_na2)
            count_non_na = len(df_new.index) - number_na
            #print(number_na,count_non_na)
            
    return(number_na,count_non_na)

In [24]:
number_na,number_real = count_na(f)

In [25]:
#this takes a correlation matrix, and turns it into flat "pannel" data. This dataframe will then be joined
#with a dataframe to determine how many valid observations make up the correlation.
def get_pannel_corr(df):
    COR = df.corr()
    namelist=list(COR.columns)
    correlation=[]
    corr = {}
    for i in range(1,len(namelist)): # i is row number,namelist[i] is row name
        for j in range(0,i):  #j is column number, namelist[j] is column name, 
            y = (COR.loc[namelist[j],namelist[i]])
            corr.update({y:{namelist[j]:namelist[i]}})
            correlation.append(COR.loc[namelist[j],namelist[i]]) 
    return(corr)  

In [26]:
c = get_pannel_corr(v)

In [27]:
#you can access info in a dictionary using items()
dict_t = {1:'one',2:'two'}
for k, v in dict_t.items():
    print(k,v)

1 one
2 two
